In [1]:
!nvidia-smi

Thu Aug 25 13:52:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Imports

In [2]:
%%capture
!pip install transformers

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import re

import random
import torch

def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed()

import warnings
warnings.simplefilter('ignore')

import gc

from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
drive.mount('/content/drive', force_remount=False)
%cd /content/drive/MyDrive/signate_student_cup_2022

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/signate_student_cup_2022


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2-xl"# gpt2-xl
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# Main Exp

In [6]:

"""
Data generation
"""
# 00. config class
class Config():

    def __init__(self, input_txts, input_tokenized):
        # length [50, 180, 60]
        _min_length = 60
        _max_length = 140
        _step_length = 40
        self.VALIATION_LENGTH = [ l for l in range(_min_length, _max_length+1, _step_length)]

        # 温度 低温(e.g 0.5)ほど一貫性, 高音(e.g 2.0)ほど多様性 >> 温度は高くしつつ文脈場あまりに奇妙な単語は除外する >> そのためのtop-p, top-k
        _min_temperature = 1.0
        _max_temperature = 1.9
        _step_temperature = 0.3
        _c = int(((_max_temperature - _min_temperature) // _step_temperature) + 1)
        self.VALIATION_TEMPERATURE = [_min_temperature+(_step_temperature*c) for c in range(_c) ]

        # 0 < p < 1.0, 低いほど多様, 温度を高く設定してpを高くして突飛な単語を抑えるか
        _min_p = 0.80
        _max_p = 0.96# 0.95
        _step_p = 0.04
        _c = int(((_max_p - _min_p) // _step_p) + 1)
        self.VALIATION_P = [ _min_p+(_step_p*c) for c in range(_c) ]

        # k 高いほど多様 [50, 150, 50]
        _min_k = 80
        _max_k = 120
        _step_k = 40
        self.VALIATION_K = [ k for k in range(_min_k, _max_k+1, _step_k)]

        # repetition_penalty
        _min_repetition_penalty = 1.0
        _max_repetition_penalty = 2.0
        _step_repetition_penalty = 0.5
        _c = int(((_max_repetition_penalty - _min_repetition_penalty) // _step_repetition_penalty) + 1)
        self.VALIATION_REPETITION_PENALTY = [ _min_repetition_penalty+(_step_repetition_penalty*c) for c in range(_c) ]

        # num_return_sequence
        self.num_return_sequence = 1

        # input_txts > とりあえず各labelでsample数は揃えておく
        self.input_txts = input_txts
        self.input_tokenized = input_tokenized

        self.len_each_txts = []
        for i in range(4):
            self.len_each_txts.append(len(input_txts[i]))

        max_sample = len(self.VALIATION_LENGTH)*len(self.VALIATION_TEMPERATURE)*len(self.VALIATION_P)*len(self.VALIATION_K)*len(self.VALIATION_REPETITION_PENALTY) * self.num_return_sequence
        self.each_sample = [max_sample//2, max_sample, max_sample//4, max_sample//4]
        self.all_sample = sum(self.each_sample)

In [7]:
# 01. create input text for each label ======================
def create_input_texts():

    input_txts = []# shape: [4, 各label毎のinput_txt数]
    input_tokenized = []
    job_names = ["data scientst", "engineer", "software engineer", "consultant"]

    # DS > 0
    ds_txt = []
    name = job_names[0]
    txt = f'The {name}\'s duties include'
    ds_txt.append(txt)

    # ME > 1
    me_txt = []
    name = job_names[1]
    txt = f'The AI {name}\'s duties include'
    me_txt.append(txt)
    txt = f'The Machine Learning {name}\'s duties include'
    me_txt.append(txt)


    # SE > 2
    se_txt = []
    name = job_names[2]
    txt = f'The {name}\'s duties include'
    se_txt.append(txt)

    # CO > 3
    co_txt = []
    name = job_names[3]
    txt = f'The IT {name}\'s duties include'
    co_txt.append(txt)
    txt = f'The {name}\'s duties include'
    co_txt.append(txt)

    # txts用意
    txts = []
    txts.append(ds_txt)
    txts.append(me_txt)
    txts.append(se_txt)
    txts.append(co_txt)

    for i in range(4):
        input_txt = []
        input_tok = []
        for txt in txts[i]:
            input_txt.append(txt)
            tokenized = tokenizer(txt, return_tensors="pt")
            input_tok.append(tokenized)
        input_txts.append(input_txt)
        input_tokenized.append(input_tok)

    config = Config(input_txts, input_tokenized)

    return config

def download_input_texts(path):
    input_txts = []
    # downloadしてそれぞれlabel毎に入れる
    return input_txts

config = create_input_texts()

In [8]:
print("length: ", config.VALIATION_LENGTH)
print("temperature: ", config.VALIATION_TEMPERATURE)
print("p: ", config.VALIATION_P)
print("k: ", config.VALIATION_K)
print("repetition_penalty: ", config.VALIATION_REPETITION_PENALTY)
print("num_return_sequence: ", config.num_return_sequence)
print("each_sample: ", config.each_sample)
print("all_sample: ", config.all_sample)

length:  [60, 100, 140]
temperature:  [1.0, 1.3, 1.6]
p:  [0.8, 0.8400000000000001, 0.88, 0.92]
k:  [80, 120]
repetition_penalty:  [1.0, 1.5, 2.0]
num_return_sequence:  1
each_sample:  [108, 216, 54, 54]
all_sample:  432


In [9]:
all_descriptions = []
all_labels = []
# debug
all_len = []
all_temp = []
all_p = []
all_k = []
all_rep_penalty = []
all_sample_txt = []

for label in range(4):

    print(f'\n===== Start Label={label} =====')
    txts = config.input_txts[label]
    tokenized = config.input_tokenized[label]
    labels = []
    descriptions = []

    for i in range(int(config.each_sample[label] / config.num_return_sequence)):

        sample_id = np.random.randint(0,len(txts))
        sample_txt = txts[sample_id]
        sample_tokenized = tokenized[sample_id]

        input_ids = sample_tokenized["input_ids"].to(device)
        attention_mask = sample_tokenized["attention_mask"].to(device)

        length = config.VALIATION_LENGTH[ np.random.randint(0, len(config.VALIATION_LENGTH)) ]
        temperature = config.VALIATION_TEMPERATURE[ np.random.randint(0, len(config.VALIATION_TEMPERATURE)) ]
        p = config.VALIATION_P[ np.random.randint(0, len(config.VALIATION_P)) ]
        k = config.VALIATION_K[ np.random.randint(0, len(config.VALIATION_K)) ]
        repetition_penalty = config.VALIATION_REPETITION_PENALTY[ np.random.randint(0, len(config.VALIATION_REPETITION_PENALTY)) ]
        num_return_sequence = config.num_return_sequence

        # debug >>>>>>>>>>>>>>>>>>>>>>>>
        print(f'i >> {i}')
        print("length: ", length)
        print("temperature: ", temperature)
        print("p: ", p)
        print("k: ", k)
        print("repetition_penalty: ", repetition_penalty)
        # debug >>>>>>>>>>>>>>>>>>>>>>>>

        outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=length, do_sample=True, 
                                      top_p=p, top_k=k, temperature=temperature, num_return_sequences=num_return_sequence, 
                                      repetition_penalty =repetition_penalty , early_stopping=True)

        for output in outputs:
            print(" - - - - - - - - - - - - - ")
            description = tokenizer.decode(output, skip_special_tokens=True)
            # header分解
            description = re.sub(sample_txt, '', description).strip()

            description = re.sub(r'data scientist', 'one', description).strip()
            description = re.sub(r'Data scientist', 'one', description).strip()
            description = re.sub(r'Data Scientist', 'one', description).strip()

            description = re.sub(r'machine learning engineer', 'one', description).strip()
            description = re.sub(r'Machine learning engineer', 'one', description).strip()
            description = re.sub(r'Machine Learning engineer', 'one', description).strip()
            description = re.sub(r'Machine Learning Engineer', 'one', description).strip()

            description = re.sub(r'AI engineer', 'one', description).strip()
            description = re.sub(r'AI Engineer', 'one', description).strip()

            description = re.sub(r'software engineer', 'one', description).strip()
            description = re.sub(r'Software engineer', 'one', description).strip()
            description = re.sub(r'Software Engineer', 'one', description).strip()

            description = re.sub(r'consultant', 'one', description).strip()
            description = re.sub(r'Consultant', 'one', description).strip()

            lines = description.split("\n")
            clean_lines = []
            for line in lines:
                clean_line = line.strip()
                clean_line = clean_line + ('' if clean_line.endswith('.') else '.')
                if len(clean_line)!=1 and len(re.sub(r'[^a-zA-Z]', '', line))!=0:
                    clean_lines.append(clean_line)
            
            if len(clean_lines) > 0:
                description = ' '.join(clean_lines)
                print(description)
                descriptions.append(description)
                labels.append(label+1)
                all_len.append(length)
                all_temp.append(temperature)
                all_p.append(p)
                all_k.append(k)
                all_rep_penalty.append(repetition_penalty)
                all_sample_txt.append(sample_txt)
                

        print("-------------------------------------------------------\n")
    all_descriptions.append(descriptions)
    all_labels.append(labels)

import itertools
all_descriptions = list(itertools.chain.from_iterable(all_descriptions))
all_labels = list(itertools.chain.from_iterable(all_labels))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



===== Start Label=0 =====
i >> 0
length:  140
temperature:  1.0
p:  0.88
k:  80
repetition_penalty:  1.0
 - - - - - - - - - - - - - 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ensuring the production of high quality, well-written and thoroughly researched research papers. There are numerous activities which could help you to be productive, but this one is probably the most critical. In order to achieve your own best research and research quality, you have to be very organized. If you do not know where to start, do not get stressed out and waste time with the usual. Try to find a research paper writing service and make a list of your key research topics and research topics you have always wanted to explore. You should write a number of such articles, and then submit the list to the service. You will get your research articles, and the service will.
-------------------------------------------------------

i >> 1
length:  60
temperature:  1.6
p:  0.8400000000000001
k:  80
repetition_penalty:  2.0


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

 - - - - - - - - - - - - - 
"reviewing an array of issues that affect the overall operation and development management" at RIT. He is also tasked with providing advice to students, faculty members or staff regarding a number of areas such as housing arrangements in collaboration "with residence advisors," on programs offered through EOCs/EPM schools (Edison House) including student services; class scheduling and how accommodations are provided for specific groups within campus communities like those targeted by racism which he has addressed online.
-------------------------------------------------------

i >> 51
length:  100
temperature:  1.3
p:  0.8
k:  120
repetition_penalty:  1.0


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 - - - - - - - - - - - - - 
troubleshooting network failures, troubleshooting applications, and developing software solutions to increase productivity, speed the deployment of network-based solutions, or improve system functionality, availability and security. Information technology staff typically will provide assistance to the CEO, business department managers, or general IT staff of the organization, and may be employed in IT-related positions or may perform other IT functions in the organization. The business analyst's responsibilities include analyzing the business processes, estimating the time required.
-------------------------------------------------------

i >> 52
length:  140
temperature:  1.6
p:  0.92
k:  80
repetition_penalty:  2.0


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 - - - - - - - - - - - - - 
the implementation of all computer technologies including network monitoring and virtualization to maximize efficiency, cost savings for governments around Asia. By bringing technological expertise along with expert marketing skills he helps build customer databases from raw data gathered by staff at his country offices who often travel across nations collecting information during day long expeditions against major multinational players located inside this destination market! The Philippines is the only Latin American nation targeted primarily on markets such as Malaysia/BVI since economic policies enacted after World War II also offer them attractive favorable trading status that greatly expand their own competitiveness.". For sales people in countries facing a large variety international competition or multiple suppliers whose efforts were just too varied — be they Chinese.
-------------------------------------------------------

i >> 53
length:  140
tempe

In [10]:
len(all_descriptions), len(all_labels), len(all_len), len(all_temp), len(all_p), len(all_k), len(all_rep_penalty), len(all_sample_txt)

(429, 429, 429, 429, 429, 429, 429, 429)

In [11]:
df = pd.DataFrame(list(zip(all_descriptions, all_labels, all_len, all_temp, all_p, all_k, all_rep_penalty, all_sample_txt)), columns = ['description','jobflag', 'len', 'temperature', 'top_p', 'top_k', 'repetition_penalty', 'input_txt'])
df

,description,jobflag,len,temperature,top_p,top_k,repetition_penalty,input_txt
0,"ensuring the production of high quality, well-...",1,140,1.0,0.88,80,1.0,The data scientst's duties include
1,"collecting, preserving and interpreting large ...",1,60,1.6,0.84,80,2.0,The data scientst's duties include
2,determining where the issues come into play to...,1,140,1.6,0.92,80,2.0,The data scientst's duties include
3,collecting and analysing research findings. In...,1,100,1.0,0.84,120,1.5,The data scientst's duties include
4,both data management and visualization on a hi...,1,100,1.3,0.92,120,1.0,The data scientst's duties include
...,...,...,...,...,...,...,...,...
424,managing the technology infrastructure for sev...,4,140,1.0,0.80,80,2.0,The IT consultant's duties include
425,"""reviewing an array of issues that affect the ...",4,100,1.0,0.84,120,2.0,The consultant's duties include
426,"troubleshooting network failures, troubleshoot...",4,100,1.3,0.80,120,1.0,The IT consultant's duties include
427,the implementation of all computer technologie...,4,140,1.6,0.92,80,2.0,The IT consultant's duties include


In [12]:
df.to_csv('data/augmented_data2.csv')